In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import pandas as pd
import hvplot.pandas
import hvplot.xarray

In [ ]:
varname = 'salinity'

In [ ]:
file=f'../tests/data/m1_hello_schism/outputs/{varname}_2.nc'

In [ ]:
ds = xr.open_dataset(file)

In [ ]:
ds

In [ ]:
#!conda install -y -c cadwr-dms schimpy

In [ ]:
import schimpy

from schimpy import schism_mesh

In [ ]:
import holoviews as hv
from holoviews import opts, dim

In [ ]:
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import datashade, rasterize
import warnings
warnings.filterwarnings('ignore')

In [ ]:
smesh = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

In [ ]:
smesh.n_nodes(), smesh.n_edges(), smesh.n_elems()

In [ ]:
varname

In [ ]:
nodes = pd.DataFrame(smesh.nodes,columns=['x','y','z'])
nodes
nodes[varname] = ds[varname].values[0,:,0]

In [ ]:
trimesh = hv.TriMesh((smesh.elems, hv.Points(nodes, vdims=varname)))

In [ ]:
trimesh = trimesh.opts(
    opts.TriMesh(cmap='rainbow4', cnorm='eq_hist', colorbar=True, node_alpha=0, edge_alpha=0, edge_color=varname, filled=True, height=400, 
                 inspection_policy='edges', tools=['hover'], width=400))

In [ ]:
trimesh.nodes.data

In [ ]:
import datashader

In [ ]:
def update(time=0, depth=0):
    trimesh.nodes.data[varname] = ds[varname].values[time, :, depth]
    return rasterize(trimesh, precompute=False, dynamic=True, 
                     aggregator=datashader.mean(varname)).opts(
        colorbar=True, cmap='rainbow4', # clim=(0,10), 
        width=600, tools=['hover'])

In [ ]:
import panel as pn
pn.extension()

In [ ]:
time_slider= pn.widgets.DiscreteSlider(name='Time',
                                       options=dict(zip(ds.time.astype('str').values,
                                                        range(len(ds.time)))))
depth_slider= pn.widgets.Select(name='nSCHISM_vgrid_layers',
                                options=dict(zip(ds.nSCHISM_vgrid_layers.astype('str').values, 
                                                 range(len(ds.nSCHISM_vgrid_layers)))))

In [ ]:
slider_row = pn.Row(time_slider, depth_slider)

In [ ]:
map_row = pn.Row(pn.bind(update, time=time_slider, depth=depth_slider))

In [ ]:
dash = pn.Column(slider_row, map_row).servable(f'HelloSCHISM: {varname}')

In [ ]:
dash.show()